<a href="https://colab.research.google.com/github/Tharini737/projects/blob/master/conformer_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/sooftware/conformer.git

Cloning into 'conformer'...
remote: Enumerating objects: 457, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 457 (delta 12), reused 8 (delta 4), pack-reused 423
Receiving objects: 100% (457/457), 2.88 MiB | 21.38 MiB/s, done.
Resolving deltas: 100% (256/256), done.


In [ ]:
!cd /content/conformer/

In [ ]:
!pwd

/content


In [ ]:
!pip install  -e ./conformer/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/conformer
  Running setup.py develop for conformer


In [ ]:
import torch
import torch.nn as nn
from conformer.conformer import Conformer

batch_size, sequence_length, dim = 3, 12345, 80

cuda = torch.cuda.is_available()  
device = torch.device('cuda' if cuda else 'cpu')

criterion = nn.CTCLoss().to(device)

inputs = torch.rand(batch_size, sequence_length, dim).to(device)
input_lengths = torch.LongTensor([12345, 12300,1000])

targets = torch.LongTensor([[1, 3, 3, 3, 3, 3, 4, 5, 6, 2],
                            [1, 3, 3, 3, 3, 3, 4, 5, 2, 0],
                            [1, 3, 3, 3, 3, 3, 4, 2, 0, 0]]).to(device)
target_lengths = torch.LongTensor([9, 8, 7])


In [ ]:
model = Conformer(num_classes=10, 
                  input_dim=dim, 
                  encoder_dim=32, 
                  num_encoder_layers=3).to(device)

# Forward propagate
outputs, output_lengths = model(inputs, input_lengths)

# Calculate CTC Loss
loss = criterion(outputs.transpose(0, 1), targets, output_lengths, target_lengths)

In [ ]:
print(outputs.shape, outputs.transpose(0,1).shape,output_lengths.shape, target_lengths.shape)

torch.Size([3, 3085, 10]) torch.Size([3085, 3, 10]) torch.Size([3]) torch.Size([3])


In [ ]:
print(outputs)

tensor([[[-3.9343, -1.1593, -2.6450,  ..., -4.0512, -2.0943, -4.5687],
         [-3.4203, -1.6311, -2.5078,  ..., -3.2894, -2.9125, -4.2375],
         [-3.9996, -0.9065, -2.5113,  ..., -2.2303, -2.6894, -3.8892],
         ...,
         [-2.0930, -2.5510, -3.1367,  ..., -1.6928, -3.4459, -2.6721],
         [-3.0943, -2.3495, -3.7359,  ..., -1.7334, -4.4423, -2.1111],
         [-2.5184, -2.1186, -3.0938,  ..., -1.7797, -3.4924, -2.5116]],

        [[-3.5571, -1.5678, -2.4107,  ..., -3.3190, -2.9047, -3.8907],
         [-3.0705, -1.9838, -2.1659,  ..., -2.7634, -2.8879, -2.8979],
         [-3.5565, -1.5389, -1.6076,  ..., -2.1244, -2.9915, -4.2800],
         ...,
         [-3.3958, -1.2155, -3.0456,  ..., -1.4087, -3.8165, -3.0483],
         [-2.9236, -1.0875, -3.6641,  ..., -1.8405, -3.5033, -3.1984],
         [-1.7861, -1.0118, -2.8049,  ..., -3.1219, -2.7466, -3.7442]],

        [[-4.2012, -1.3934, -1.9568,  ..., -2.4689, -2.2525, -4.3206],
         [-3.9478, -1.0753, -1.8750,  ..., -2

In [ ]:
print(output_lengths, target_lengths)

tensor([3085, 3074,  249]) tensor([9, 8, 7])


In [ ]:
from torch.utils.data import DataLoader, Dataset
import numpy as np
class TimeSeriesDataset(Dataset):
    r"""Pytorch Dataset object
    Args:
        Dataset (torch.utils.data.Dataset): Pyptorch dataset object
    """

    def __init__(
        self,
        data,
    ):
        self.data = data

    def __getitem__(self, index):
        data = self.data[index]    
        return data, 

    def __len__(self):
        return len(self.data)

In [ ]:
def sliding_windows(data, target, seq_length):
    x = []
    y = []
    for i in range(len(data)-seq_length):
        _x = data[i:(i+seq_length)]
        _y = target[i]
        x.append(_x)
        y.append(_y)
    return np.array(x), np.array(y)


In [ ]:
input_len = 10
input_dim = 2
data = np.random.randn(input_len,input_dim) # Some random data (1, timesteps) or ( timesteps, 1)
target = np.array([0,1,2,3,4,5,6,7,8,9])

In [ ]:
data.shape

(10, 2)

In [ ]:
data, target = sliding_windows(data = data,target=target, seq_length = 2)

In [ ]:
data.shape # (number_of_sequences/total number of batches, sequence length, input_dim)

(8, 2, 2)

In [ ]:
target.shape

(8,)

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset = data,batch_size= 16)
i = 0
for z in dataloader:
    print(z.shape)
    i+=1
    print(i)

torch.Size([16, 10, 5])
1
torch.Size([16, 10, 5])
2
torch.Size([16, 10, 5])
3
torch.Size([16, 10, 5])
4
torch.Size([16, 10, 5])
5
torch.Size([16, 10, 5])
6
torch.Size([16, 10, 5])
7
torch.Size([16, 10, 5])
8
torch.Size([16, 10, 5])
9
torch.Size([16, 10, 5])
10
torch.Size([16, 10, 5])
11
torch.Size([16, 10, 5])
12
torch.Size([16, 10, 5])
13
torch.Size([16, 10, 5])
14
torch.Size([16, 10, 5])
15
torch.Size([16, 10, 5])
16
torch.Size([16, 10, 5])
17
torch.Size([16, 10, 5])
18
torch.Size([16, 10, 5])
19
torch.Size([16, 10, 5])
20
torch.Size([16, 10, 5])
21
torch.Size([16, 10, 5])
22
torch.Size([16, 10, 5])
23
torch.Size([16, 10, 5])
24
torch.Size([16, 10, 5])
25
torch.Size([16, 10, 5])
26
torch.Size([16, 10, 5])
27
torch.Size([16, 10, 5])
28
torch.Size([16, 10, 5])
29
torch.Size([16, 10, 5])
30
torch.Size([16, 10, 5])
31
torch.Size([16, 10, 5])
32
torch.Size([16, 10, 5])
33
torch.Size([16, 10, 5])
34
torch.Size([16, 10, 5])
35
torch.Size([16, 10, 5])
36
torch.Size([16, 10, 5])
37
torch.Size

In [ ]:
next(iter(dataloader)).shape

torch.Size([16, 10, 5])